In [168]:
import numpy as np
import pandas as pd
import math

In [169]:
articles = pd.read_csv('data/shared_articles.csv')
articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [170]:
shared_arcticles = articles[articles['eventType'] == 'CONTENT SHARED']

In [171]:
interactions = pd.read_csv('data/users_interactions.csv')
interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [172]:
# Назначение весов для типов взаимодействия
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [173]:
# Определение числового веса для взаимодействия со статьёй
interactions['value'] = interactions['eventType'].apply(lambda x: event_type[x])

In [174]:
# Определение только тех пользователей, которые взаимодействовали хотя бы с пятью статьями
users_df = (
    interactions
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_5_df = users_df[users_df >= 5].reset_index()[['personId']]
print(len(users_5_df))

1140


In [175]:
# Отфильтруем пользователей, которые взаимодействовали как минимум с пятью статьями
int_from_users5_df = interactions.loc[np.in1d(interactions.personId,
            users_5_df)] #этот метод используется, если мы хотим найти  пересечение двух одномерных массивов NumPy
print(int_from_users5_df.shape)

(69868, 9)


In [176]:
# Функция логарифмирования суммы весов взаимодействий 
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [177]:
# Применим функцию логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. 
# Также сохраним для каждой пары «пользователь — статья» значение времени последнего взаимодействия.
full_df = (
    int_from_users5_df
    .groupby(['personId', 'contentId'])['value'].sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)

full_df['last_timestamp'] = (
    int_from_users5_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
full_df['last_timestamp'].mean()

1470587338.35212

In [178]:
# Разделение данных на тренировочную и тестовую выборки
train_df = full_df.loc[full_df.last_timestamp < 1475519545].copy()
test_df = full_df.loc[full_df.last_timestamp >= 1475519545].copy()

In [179]:
# Для удобства дальнейшего измерения качества рекомендаций преобразуем данные так, чтобы получить таблицу в формате, 
# где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. 
# На место пустых ячеек поместим пустые списки.
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]

In [180]:
final_df.head()

,true_train,true_test
personId,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[]
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084]
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]"
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -..."


In [181]:
# Оценка популярности каждой статьи как суммы всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку)
popular = (
    train_df
    .groupby('contentId')
    .value.sum().reset_index()
    .sort_values('value', ascending=False)
    .contentId.values
)

print(f'Id самой популярной статьи по сумме логарифмических оценок взаимодействий: {popular[0]}')

Id самой популярной статьи по сумме логарифмических оценок взаимодействий: -6783772548752091658


In [182]:
# Сформируем рекомендации для каждого пользователя. 
# Будем рекомендовать десять самых популярных статей. 
# Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)


In [183]:
# Оценка качества с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). 
# После этого усреднение результата по всем пользователям.
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()


print(f'Оценка точности постоенной рекомендательной системы: {round(precision("popular"), 3)}')

Оценка точности постоенной рекомендательной системы: 0.006


Стоит отметить, что качество РС оценивается не так, как в задачах классификации: показателей выше 0.5 добиться практически невозможно, и даже результат 0.1–0.2 — индикатор высокого качества.